In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
from torch import nn

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [14]:
double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mixtral-8x7B-v0.1", quantization_config=double_quant_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

AttributeError: 'LlamaTokenizerFast' object has no attribute 'to'

In [15]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

In [20]:
activation = {}


def getActivation(name):
    # the hook signature
    def hook(model, input, output):
        activation[name] = output.detach()

    return hook

h = model.model.layers[15].block_sparse_moe.experts[7].w3.register_forward_hook(getActivation('15_7'))
prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
print(tokenizer.batch_decode(generated_ids)[0])

print(activation['15_7'].shape)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<class 'transformers.tokenization_utils_base.BatchEncoding'>
<s> My favourite condiment is mustard- from hot to brown, all varieties are welcome at my table. If I had to choose, only one would win at the top of my list because it is so versatile – dijon mustard. I use it to make a variety of dips and sauces to liven up meats, veggies and even add it into savoury baked goods.

This sauce is a great one to have around if your little ones don’t like salad dress
torch.Size([1, 14336])


In [7]:
print(activation["15_7"])

tensor([[ 0.0778,  0.4185, -1.3171,  ...,  0.3009,  0.6739,  0.3222]])


In [ ]:
h = (
    model.model.layers[15]
    .block_sparse_moe.experts[7]
    .w3.register_forward_hook(getActivation("15_7"))
)

experts = model.model.layers[15].block_sparse_moe.experts

for expert_idx in experts:
    experts[expert_idx].w3.register_forward_hook(getActivation(f"layer_15_expert_{expert_idx}"))

In [62]:
# i = 0
# for child in model.children().:
#     # for param in child.parameters():
#     print(child)
#     i+=1
#     print(i)
#     # break
# top_layers = list(model.children())
# print(top_layers)
# named_layers = dict(model.named_modules())
# print(named_layers)
# # model.children()

dict(list(list(list(model.children())[0].children())[1])[15])

TypeError: 'MixtralDecoderLayer' object is not iterable

In [9]:
model.model.layers[15].block_sparse_moe.experts[7].w3.weight

Parameter containing:
tensor([[-0.0167,  0.0057, -0.0010,  ..., -0.0171, -0.0109,  0.0067],
        [ 0.0073,  0.0181, -0.0004,  ...,  0.0011, -0.0033, -0.0107],
        [-0.0001, -0.0084, -0.0028,  ...,  0.0074,  0.0011, -0.0103],
        ...,
        [ 0.0014,  0.0029, -0.0205,  ...,  0.0079, -0.0149, -0.0023],
        [-0.0020, -0.0045,  0.0031,  ..., -0.0280, -0.0013, -0.0032],
        [-0.0095, -0.0057, -0.0022,  ..., -0.0254, -0.0029, -0.0134]],
       requires_grad=True)